<a href="https://colab.research.google.com/github/andreaderuvo/rc_car_tracking/blob/main/datasets/extract_random_frames_from_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#folder where to upload videos manually or from Google Drive
INPUT_VIDEOS_FOLDER = 'videos'
OUTPUT_IMAGES_FOLDER = 'images'
MAX_NUMBER_OF_FRAMES_PER_VIDEO = 100

In [ ]:
import cv2
import os, datetime
import random
from os import listdir
import shutil
import time

In [ ]:
#check if the execution environment is Google Colab
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
# Empty folder OUTPUT_IMAGES_FOLDER and INPUT_VIDEOS_FOLDER
shutil.rmtree(INPUT_VIDEOS_FOLDER, ignore_errors=True)
shutil.rmtree(OUTPUT_IMAGES_FOLDER, ignore_errors=True)

In [ ]:
# create folder
os.makedirs(INPUT_VIDEOS_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_IMAGES_FOLDER, exist_ok=True)

In [ ]:
#copy downloaded videos from Google Drive to INPUT_FOLDER
#skip this if you upload videos manually in INPUT_FOLDER
DRIVE_MOUNT_FOLDER = 'drive'
DRIVE_VIDEO_FOLDER = 'MyDrive/rc_car_tracking/videos'

if IN_COLAB:
  from google.colab import drive
  import shutil
  drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)
  google_drive_video_folder = os.path.join(DRIVE_MOUNT_FOLDER, DRIVE_VIDEO_FOLDER)
  for file in os.listdir(google_drive_video_folder):
    file_path = os.path.join(google_drive_video_folder, file)
    shutil.copy(file_path, INPUT_VIDEOS_FOLDER)
  drive.flush_and_unmount()

In [ ]:
# Interval of frames considered in percentages (0 -> 0%, 1->100%)
P_FRAMES = (0.2,0.8)

# Extract random images in OUTPUT_IMAGES_FOLDER from each video in INPUT_VIDEOS_FOLDER
videos = os.listdir(INPUT_VIDEOS_FOLDER)

for i,video in enumerate(videos):
  
  if not video.endswith(".mp4"):
    continue;

  cap = cv2.VideoCapture(f"{INPUT_VIDEOS_FOLDER}/{video}")

  if not cap.isOpened(): 
    print(f'Could not open video {video}, {i+1}/{len(videos)}')
    break;

  length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  print(f'Processing video {video} with {length} frames, {i+1}/{len(videos)}')

  deck = list(range(int(length * P_FRAMES[0]), int(length * P_FRAMES[1])))

  if len(deck) < MAX_NUMBER_OF_FRAMES_PER_VIDEO:
    print(f'Video {video} does not contain enough frames ({length} total frames) to extract in according to MAX_NUMBER_OF_FRAMES_PER_VIDEO ({MAX_NUMBER_OF_FRAMES_PER_VIDEO}) set')
    break;

  random.shuffle(deck)  

  num_image = 0      
  counter = 0;

  while num_image < MAX_NUMBER_OF_FRAMES_PER_VIDEO:
    num_frame = deck.pop()
    cap.set(cv2.CAP_PROP_POS_FRAMES, num_frame - 1)
    success, image = cap.read()
    if success:
      file_name = f'{i + 1}_{num_image}_{num_frame}.jpg'
      path = os.path.join(OUTPUT_IMAGES_FOLDER, file_name)
      cv2.imwrite(path, image)
      print(f"\rExtracting random frame {counter + 1} of {MAX_NUMBER_OF_FRAMES_PER_VIDEO}", end='', flush=True) 
      num_image += 1
      counter += 1
    else:
      print(f'Error in reading frame {num_frame}...')

  print('\n')

In [ ]:
#download locally images in a zip file from Google Colab
if IN_COLAB:
  shutil.make_archive(f'{OUTPUT_IMAGES_FOLDER}', 'zip', OUTPUT_IMAGES_FOLDER)
  from google.colab import files
  files.download(f'{OUTPUT_IMAGES_FOLDER}.zip')

In [ ]:
#upload images in Google Drive
DRIVE_MOUNT_FOLDER = 'drive'
DRIVE_UPLOAD_FOLDER = 'MyDrive/rc_car_tracking/images'

if IN_COLAB:
  from google.colab import drive
  import shutil
  drive.mount(DRIVE_MOUNT_FOLDER, force_remount=True)
  googe_drive_folder = os.path.join(DRIVE_MOUNT_FOLDER, DRIVE_UPLOAD_FOLDER)
  os.makedirs(googe_drive_folder, exist_ok=True)
  for file in os.listdir(OUTPUT_IMAGES_FOLDER):
    shutil.copy(os.path.join(OUTPUT_IMAGES_FOLDER, file), googe_drive_folder)
  drive.flush_and_unmount()
